<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

In [3]:
import numpy as np

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


import json 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library



Solving environment: done

## Package Plan ##

  environment location: /Users/marcosnegresaura/opt/anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.8.0                |           py37_1         3.0 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following packages will be UPDATED:

  conda                                        4.8.0-py37_0 --> 4.8.0-py37_1



conda-4.8.0          | 3.0 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

# All requested packages already installed.



## Project description

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download and Explore Dataset</a>

2. <a href="#item2">Clean Data</a>

3. <a href="#item2">Explore Neighbourhoods in Toronto</a>

4. <a href="#item3">Analyze Each Neighbourhood</a>

5. <a href="#item4">Cluster Neighbourhoods</a>

6.  <a href="#item5">Examine Clusters</a>    
</font>
</div>

## 1. Download and Explore Dataset

In [336]:
Data_health=pd.read_html('https://en.wikipedia.org/wiki/List_of_countries_by_total_health_expenditure_per_capita')[5]
Data_health.head(190)

,Countries,2000,2005,2010,2015
0,Afghanistan,:,24,46,60
1,Albania,76,166,204,266
2,Algeria,61,100,228,292
3,Andorra,2051,4037,4596,4316
4,Angola,14,57,97,109
5,Antigua and Barbuda,381,490,673,657
6,Argentina,418,327,699,998
7,Armenia,41,116,169,366
8,Australia,1632,3004,4953,4934
9,Austria,2263,3676,4725,4536


In [337]:
Data_health.shape

(189, 5)

In [338]:

Data_Cost_Life=pd.read_html('https://www.numbeo.com/cost-of-living/rankings_by_country.jsp')[2]
Data_Cost_Life.head(140)

,Rank,Country,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index
0,NaN,Cayman Islands,141.64,74.40,109.49,157.93,105.03,137.88
1,NaN,Bermuda,138.22,87.62,114.03,124.87,150.40,79.87
2,NaN,Switzerland,122.67,50.22,88.03,121.29,124.04,127.76
3,NaN,Norway,104.49,37.26,72.34,93.94,115.59,98.00
4,NaN,Us Virgin Islands,97.23,48.24,73.81,87.42,90.74,54.58
5,NaN,Iceland,97.22,45.08,72.29,84.63,111.46,92.03
6,NaN,Bahamas,85.96,35.40,61.79,68.84,87.20,56.21
7,NaN,Japan,85.52,28.08,58.06,84.69,50.05,97.57
8,NaN,Luxembourg,84.68,55.44,70.70,71.60,92.91,107.89
9,NaN,Denmark,83.88,33.73,59.90,64.02,100.63,110.69


In [339]:
Data_Cost_Life.shape

(136, 8)

In [340]:
Data_Life_Expentancy=pd.read_html('https://en.wikipedia.org/wiki/List_of_countries_by_life_expectancy')[1]
Data_Life_Expentancy.head(196)

Rank                     Country/Region  \
    Rank                     Country/Region   
0      1                          Hong Kong   
1      2                              Japan   
2      3                        Switzerland   
3      4                          Singapore   
4      5                              Italy   
5      5                              Spain   
6      7                          Australia   
7      8                            Iceland   
8      9                             Israel   
9      9                        South Korea   
10    11                             Sweden   
11    12                             France   
12    13                              Malta   
13    14                             Canada   
14    14                             Norway   
15    16                             Greece   
16    16                            Ireland   
17    16                         Luxembourg   
18    16                        Netherlands   
19    16                        New Zealand   
20    21                           Portugal   
21    22                            Andorra   
22    23                            Finland   
23    24                            Belgium   
24    25                            Austria   
25    26                            Germany   
26    26                           Slovenia   
27    26                     United Kingdom   
28    29                             Cyprus   
29    29                            Denmark   
30    31                      Liechtenstein   
31    32                         Costa Rica   
32    32                              Qatar   
33    34                              Chile   
34    35                            Czechia   
35    36                           Barbados   
36    37                            Lebanon   
37    37                      United States   
38    39                               Cuba   
39    40                            Estonia   
40    40                           Maldives   
41    42                            Albania   
42    42                             Poland   
43    44                            Croatia   
44    44                             Panama   
45    46                           Dominica   
46    47               United Arab Emirates   
47    47                            Uruguay   
48    49                               Oman   
49    50                           Slovakia   
50    50                             Turkey   
51    52             Bosnia and Herzegovina   
52    53                            Bahrain   
53    54                           Colombia   
54    55                Antigua and Barbuda   
55    55                           Thailand   
56    57                            Ecuador   
57    57                         Montenegro   
58    57                          Sri Lanka   
59    60                            Algeria   
60    60         People's Republic of China   
61    60                            Hungary   
62    63                          Argentina   
63    63                               Iran   
64    63                            Morocco   
65    63                               Peru   
66    63                            Tunisia   
67    68                        Saint Lucia   
68    69                           Malaysia   
69    70                            Romania   
70    71                             Serbia   
71    72                             Brazil   
72    72                             Brunei   
73    72                          Lithuania   
74    72                    North Macedonia   
75    76                             Kuwait   
76     —    Latin America and the Caribbean   
77     —          East Asia and the Pacific   
78    77                            Vietnam   
79    78                             Latvia   
80    79                           Honduras   
81    80                             Mexico   
82    80                       Saudi Arabia   
83    82 

In [341]:
Data_Life_Expentancy.shape

(196, 5)

In [342]:
Country_GeoData=pd.read_html('https://developers.google.com/public-data/docs/canonical/countries_csv')[0]
Country_GeoData.rename(columns={'name':'Country'}, inplace=True)
Country_GeoData.head(100)


,country,latitude,longitude,Country
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla
5,AL,41.153332,20.168331,Albania
6,AM,40.069099,45.038189,Armenia
7,AN,12.226079,-69.060087,Netherlands Antilles
8,AO,-11.202692,17.873887,Angola
9,AQ,-75.250973,-0.071389,Antarctica


## 2. Clean data

In [343]:
Data_health.reset_index(inplace=True)
Data_health.drop(columns=['index'], inplace=True)
Data_health.rename(columns={'Countries':'Country'}, inplace=True)
Data_health.replace(':', 0, inplace=True)
Data_health.loc[180, 'Country']='United States'
Data_health.head(200)

,Country,2000,2005,2010,2015
0,Afghanistan,0,24,46,60
1,Albania,76,166,204,266
2,Algeria,61,100,228,292
3,Andorra,2051,4037,4596,4316
4,Angola,14,57,97,109
5,Antigua and Barbuda,381,490,673,657
6,Argentina,418,327,699,998
7,Armenia,41,116,169,366
8,Australia,1632,3004,4953,4934
9,Austria,2263,3676,4725,4536


In [344]:
#Data_Cost_Life.drop(columns=['Rank'], inplace=True)
Data_Cost_Life.rename(columns={'Countries':'Country'}, inplace=True)
Data_Cost_Life.replace(':', 0, inplace=True)
Data_Cost_Life.head(100)

,Rank,Country,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index
0,NaN,Cayman Islands,141.64,74.40,109.49,157.93,105.03,137.88
1,NaN,Bermuda,138.22,87.62,114.03,124.87,150.40,79.87
2,NaN,Switzerland,122.67,50.22,88.03,121.29,124.04,127.76
3,NaN,Norway,104.49,37.26,72.34,93.94,115.59,98.00
4,NaN,Us Virgin Islands,97.23,48.24,73.81,87.42,90.74,54.58
5,NaN,Iceland,97.22,45.08,72.29,84.63,111.46,92.03
6,NaN,Bahamas,85.96,35.40,61.79,68.84,87.20,56.21
7,NaN,Japan,85.52,28.08,58.06,84.69,50.05,97.57
8,NaN,Luxembourg,84.68,55.44,70.70,71.60,92.91,107.89
9,NaN,Denmark,83.88,33.73,59.90,64.02,100.63,110.69


In [345]:
Data_Life_Expentancy.columns = Data_Life_Expentancy.columns.droplevel()
Data_Life_Expentancy.replace(':', 0, inplace=True)
Data_Life_Expentancy.reset_index(0).reset_index(drop=True)
Data_Life_Expentancy.rename(columns={'Country/Region':'Country'}, inplace=True)
Data_Life_Expentancy.head(200)

,Rank,Country,Overall,Female,Male
0,1,Hong Kong,84.7,87.6,81.8
1,2,Japan,84.5,87.5,81.3
2,3,Switzerland,83.6,85.5,81.7
3,4,Singapore,83.5,85.6,81.3
4,5,Italy,83.4,85.4,81.1
5,5,Spain,83.4,86.1,80.7
6,7,Australia,83.3,85.3,81.3
7,8,Iceland,82.9,84.4,81.3
8,9,Israel,82.8,84.4,81.1
9,9,South Korea,82.8,85.8,79.7


In [346]:

Data_health=Data_health.join(Data_Life_Expentancy.set_index('Country'), on='Country', how='left', lsuffix='_left', rsuffix='_right')
Data_health=Data_health.join(Data_Cost_Life.set_index('Country'), on='Country', how='left', lsuffix='_left', rsuffix='_right')
Data_health=Data_health.join(Country_GeoData.set_index('Country'), on='Country', how='left', lsuffix='_left', rsuffix='_right')


In [347]:
Data_health.drop(columns=['Rank_left','Female', 'Male','Rank_right', 'Rent Index','Cost of Living Plus Rent Index', 'Groceries Index', 'Restaurant Price Index', 'Restaurant Price Index', 'Local Purchasing Power Index','country','2000','2005','2010'], inplace=True)
Data_health.head(110)

,Country,2015,Overall,Cost of Living Index,latitude,longitude
0,Afghanistan,60,64.5,24.51,33.939110,67.709953
1,Albania,266,78.5,36.63,41.153332,20.168331
2,Algeria,292,76.7,30.54,28.033886,1.659626
3,Andorra,4316,81.8,NaN,42.546245,1.601554
4,Angola,109,60.8,NaN,-11.202692,17.873887
5,Antigua and Barbuda,657,76.9,NaN,17.060816,-61.796428
6,Argentina,998,76.5,38.72,-38.416097,-63.616672
7,Armenia,366,74.9,32.70,40.069099,45.038189
8,Australia,4934,83.3,73.39,-25.274398,133.775136
9,Austria,4536,81.4,72.15,47.516231,14.550072


In [348]:
Data_health.dropna(inplace=True)
Data_health.rename(columns={'2015':'Health Care Expenses','latitude':'Latitude','longitude':'Longitude'}, inplace=True)
Data_health.reset_index(inplace=True,drop=True)
Data_health.head(200)

,Country,Health Care Expenses,Overall,Cost of Living Index,Latitude,Longitude
0,Afghanistan,60,64.5,24.51,33.939110,67.709953
1,Albania,266,78.5,36.63,41.153332,20.168331
2,Algeria,292,76.7,30.54,28.033886,1.659626
3,Argentina,998,76.5,38.72,-38.416097,-63.616672
4,Armenia,366,74.9,32.70,40.069099,45.038189
5,Australia,4934,83.3,73.39,-25.274398,133.775136
6,Austria,4536,81.4,72.15,47.516231,14.550072
7,Azerbaijan,368,72.9,29.81,40.143105,47.576927
8,Bahamas,1685,73.8,85.96,25.034280,-77.396280
9,Bahrain,1190,77.2,58.36,25.930414,50.637772


In [349]:
Data_health.shape

(111, 6)

## 3. Explore neighbourhoods in Toronto

In [351]:
World_map = folium.Map(location=[0, 0], zoom_start=2)
color='blue'

# add markers to map
for lat, lng, country in zip(Data_health['Latitude'], Data_health['Longitude'] , Data_health['Country']):
    label = '{}'.format(country)
    label = folium.Popup(label, parse_html=True) 
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=color,
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(World_map)  
    
World_map

Map of Toronto with the Venues downloaded from Foursquare 

In [289]:
map_Toronto = folium.Map(location=[43.65, -79.35], zoom_start=10)
color=['blue','black','red','white']
from random import randint
# add markers to map
for lat, lng, borough, neighborhood in zip(Data_clean['Latitude'], Data_clean['Longitude'], Data_clean['Borough'], Data_clean['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    p=randint(0, 3)
      
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=color[p],
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto
#import cv2
#originalImage = cv2.imread(map_Toronto)
#cv2.imwrite('map_Toronto.jpg', map_Toronto)

NameError: name 'Data_clean' is not defined

In [290]:
CLIENT_ID = 'JMVF4CRSAGPKHIZBTLP1JASTXCWG2TXX2V54BMJ24X13HJZH' # your Foursquare ID
CLIENT_SECRET = '3S4JISGI0RES14MOR25VSBKTP22E3U4BGVJXLY4DCCP25ZOY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JMVF4CRSAGPKHIZBTLP1JASTXCWG2TXX2V54BMJ24X13HJZH
CLIENT_SECRET:3S4JISGI0RES14MOR25VSBKTP22E3U4BGVJXLY4DCCP25ZOY


In [16]:
Toronto_data.loc[0, 'Neighbourhood']

'Parkwoods'

In [17]:
neighborhood_latitude = Toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


## 4. Analize each Neighbourhood

In [18]:

LIMIT = 200 # limit of number of venues returned by Foursquare API

radius = 500 # define radius


url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/explore?&client_id=JMVF4CRSAGPKHIZBTLP1JASTXCWG2TXX2V54BMJ24X13HJZH&client_secret=3S4JISGI0RES14MOR25VSBKTP22E3U4BGVJXLY4DCCP25ZOY&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=200'

In [19]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5df616375fb726001bb4721a'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': 

In [20]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [21]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [22]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
Toronto_venues = getNearbyVenues(names=Toronto_data['Neighbourhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )




Parkwoods
Victoria Village
Harbourfront
Lawrence Heights, Lawrence Manor
Queen's Park
Queen's Park
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Highland Creek, Rouge Hill, Port Union
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale


KeyError: 'groups'

In [25]:
print(Toronto_venues.shape)
Toronto_venues.head()

NameError: name 'Toronto_venues' is not defined

In [ ]:
Toronto_venues.groupby('Neighbourhood').count()

In [ ]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

In [ ]:

Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

Toronto_onehot['Neighbourhood'] = Toronto_venues['Neighbourhood'] 

fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

In [ ]:
Toronto_onehot.shape

In [ ]:
Toronto_grouped = Toronto_onehot.groupby('Neighbourhood').mean().reset_index()


In [ ]:
Toronto_grouped.shape

In [ ]:
num_top_venues = 5

for hood in Toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

## 5. Cluster Neighbourhoods

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = Toronto_grouped['Neighbourhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

In [ ]:
kclusters = 8

Toronto_grouped_clustering = Toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
Toronto_merged = Toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

Toronto_merged.head() # check the last columns!

Down below it is desplayed the map with the different designed clusters

In [ ]:
# create map
map_clusters = folium.Map(location=[43.65, -79.35], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
Toronto_merged.dropna(inplace=True)
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 6. Examine Clusters

After clustering between three and ten groups we have decided that the optimum is found at 4 different clusters

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 5, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 6, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 7, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 8, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 9, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]